In [1]:
# imports
import sys
sys.path.append('../')

from glob import glob
from src.load_data import master_load
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
import time

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Activation, BatchNormalization, Conv1D, Conv2D,
                                     Dense, Dropout, Flatten, LSTM, MaxPooling1D,
                                     Reshape, TimeDistributed)
from tensorflow.math import confusion_matrix

In [2]:
# set plotting params
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 32
plt.rcParams['axes.labelsize'] = 32
plt.rcParams['axes.titlesize'] = 32
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams['legend.fontsize'] = 32
plt.rcParams['figure.titlesize'] = 48

In [170]:
# load image data
train_l_paths = glob('../img/data/train/left/*')
train_n_paths = glob('../img/data/train/none/*')
train_r_paths = glob('../img/data/train/right/*')
test_l_paths = glob('../img/data/test/left/*')
test_n_paths = glob('../img/data/test/none/*')
test_r_paths = glob('../img/data/test/right/*')

In [190]:
def load_image(path):
    rgba_image = Image.open(path)
    rgb_image = rgba_image.convert('RGB')
    return np.asarray(rgb_image)

In [197]:
def stack_images(paths_array):
    array = []
    
    for i in paths_array:
        array.append(load_image(i))
    
    return np.array(array)

In [201]:
left_T_images = stack_images(train_l_paths)
none_T_images = stack_images(train_n_paths)
right_T_images = stack_images(train_r_paths)

left_t_images = stack_images(test_l_paths)
none_t_images = stack_images(test_n_paths)
right_t_images = stack_images(test_r_paths)

In [203]:
combined_data = []
for action in actions:
    for data in data_dct[action]:

        if action == "left":
            combined_data.append([data, [1, 0, 0]])
        elif action == "right":
            combined_data.append([data, [0, 0, 1]])
        elif action == "none":
            combined_data.append([data, [0, 1, 0]])

(370, 1216, 280, 3)

In [219]:
def combine(left, none, right):
    combined = []
    for i in left:
        combined.append([i, [1, 0, 0]])

    for i in none:
        combined.append([i, [0, 1, 0]])

    for i in right:
        combined.append([i, [0, 0, 1]])
        
    return combined

In [218]:
X_train_img = combine()

1089

In [3]:
X_train, X_test, y_train, y_test = master_load()

In [4]:
seven_XT = X_train[:, 6, :]
fifteen_XT = X_train[:, 14, :]

seven_Xt = X_test[:, 6, :]
fifteen_Xt = X_test[:, 14, :]

In [5]:
X_train_slice = np.stack((seven_XT, fifteen_XT), axis=1)
X_test_slice = np.stack((seven_Xt, fifteen_Xt), axis=1)

In [6]:
model = Sequential()

model.add(LSTM(64, return_sequences=True, input_shape=X_train.shape[1:]))

model.add(LSTM(64, return_sequences=True))

model.add(TimeDistributed(
    Dense(32, activation='relu')))

model.add(TimeDistributed(
    Dense(8, activation='relu')))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
epochs = 50
batch_size = 32

fitted = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
score = fitted.model.evaluate(X_test, y_test, batch_size=batch_size)

model_name = f'../models/numpy/acc_{round(score[1], 3)}-loss_{round(score[0], 2)}-{int(time.time())}.model'

df = pd.DataFrame.from_dict(fitted.history)
df['model_name'] = model_name
fitted.model.save(model_name)

print('done')

Epoch 1/50
8895/8895 [==============================] - 109s 12ms/step - loss: 0.9497 - accuracy: 0.5035 - val_loss: 1.1730 - val_accuracy: 0.4186
Epoch 2/50
 797/8895 [=>............................] - ETA: 1:30 - loss: 0.7837 - accuracy: 0.6131

In [ ]:
# modified
model = Sequential()

model.add(Conv1D(64, (1), input_shape=X_train.shape[1:]))
# model.add(Conv2D(16, (3, 3), input_shape=X_train.shape[1:]))  # Test the Conv2D
model.add(Activation('selu'))

model.add(Conv1D(32, (2)))
model.add(Activation('selu'))

model.add(MaxPooling1D((2), padding='same'))
model.add(Flatten())

model.add(Dense(64))
model.add(Activation('selu'))

model.add(Dense(16))
model.add(Activation('selu'))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
epochs = 50
batch_size = 32

fitted = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
score = fitted.model.evaluate(X_test, y_test, batch_size=batch_size)

model_name = f'../models/numpy/acc_{round(score[1], 3)}-loss_{round(score[0], 2)}-{int(time.time())}.model'

temp_df = pd.DataFrame.from_dict(fitted.history)
temp_df['model_name'] = model_name
df = pd.concat([df, temp_df])
fitted.model.save(model_name)

print('done')

In [ ]:
# modified
model = Sequential()

model.add(Conv1D(64, (2), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv1D(64, (1)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(64, (1)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(16))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
epochs = 50
batch_size = 32

fitted = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
score = fitted.model.evaluate(X_test, y_test, batch_size=batch_size)

model_name = f'../models/numpy/acc_{round(score[1], 3)}-loss_{round(score[0], 2)}-{int(time.time())}.model'

temp_df = pd.DataFrame.from_dict(fitted.history)
temp_df['model_name'] = model_name
df = pd.concat([df, temp_df])
fitted.model.save(model_name)

print('done')

In [ ]:
# stolen 1
model = Sequential()

model.add(Conv1D(128, (1), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))

model.add(Conv1D(63, (2)))
model.add(Activation('relu'))

model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Conv1D(64, (1)))
model.add(Activation('relu'))

model.add(MaxPooling1D(pool_size=(2), padding='same'))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(16))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
epochs = 50
batch_size = 32

fitted = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
score = fitted.model.evaluate(X_test, y_test, batch_size=batch_size)

model_name = f'../models/numpy/acc_{round(score[1], 3)}-loss_{round(score[0], 2)}-{int(time.time())}.model'

temp_df = pd.DataFrame.from_dict(fitted.history)
temp_df['model_name'] = model_name
df = pd.concat([df, temp_df])
fitted.model.save(model_name)

print('done')

In [ ]:
reshape = (-1, 2, 60)

train_X = np.clip(np.array(X_train).reshape(reshape), -10, 10) / 10
test_X = np.clip(np.array(X_test).reshape(reshape), -10, 10) / 10

In [ ]:
# stolen 2
model = Sequential()

model.add(Conv1D(128, (1), padding='same', input_shape=train_X.shape[1:]))
model.add(Activation('relu'))

model.add(Conv1D(64, (5), padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(16, (5), padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(8, (5), padding='same'))
model.add(Activation('relu'))

model.add(Conv1D(3, (2)))
model.add(Reshape((3,)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
epochs = 50
batch_size = 32

fitted = model.fit(train_X, y_train, batch_size=batch_size, epochs=epochs, validation_data=(test_X, y_test))
score = fitted.model.evaluate(test_X, y_test, batch_size=batch_size)

model_name = f'../models/numpy/acc_{round(score[1], 3)}-loss_{round(score[0], 2)}-{int(time.time())}.model'

temp_df = pd.DataFrame.from_dict(fitted.history)
temp_df['model_name'] = model_name
df = pd.concat([df, temp_df])
fitted.model.save(model_name)

print('done')

In [ ]:
def residual_block(x, filters, conv_num=3, activation="relu"):
    # Shortcut
    s = keras.layers.Conv1D(filters, 1, padding="same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding="same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPool1D(pool_size=2, strides=2, padding='same')(x)

In [ ]:
def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape, name="input")

    x = residual_block(inputs, 16, 2)
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 3)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.MaxPooling1D(pool_size=(2), padding='same')(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation="softmax", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)

In [ ]:
model = build_model((X_train.shape[1:]), 3)

In [ ]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
epochs = 50
batch_size = 32

fitted = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))
score = fitted.model.evaluate(X_test, y_test, batch_size=batch_size)

model_name = f'../models/numpy/acc_{round(score[1], 3)}-loss_{round(score[0], 2)}-{int(time.time())}.model'

temp_df = pd.DataFrame.from_dict(fitted.history)
temp_df['model_name'] = model_name
df = pd.concat([df, temp_df])
fitted.model.save(model_name)

print('done')

In [ ]:
best_model = sorted(glob('../models/numpy/*.model'))[-1]

In [ ]:
model = tf.keras.models.load_model(best_model)

y_pred = np.argmax(model.predict(X_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)
conf_mat = np.round(np.array(confusion_matrix(y_true, y_pred)) / 11750, 3)

In [ ]:
def plot_conf_mat(conf_mat):
    actions = ['left', 'none', 'right']

    fig, ax = plt.subplots(figsize=(16, 9))

    ax.matshow(conf_mat, cmap=plt.cm.RdYlGn)
    ax.set_xticklabels([""]+actions)
    ax.set_yticklabels([""]+actions)

    for i, val0 in enumerate(conf_mat.T):
        for ii, val1 in enumerate(val0):
            ax.text(i, ii, f"{val1}", va='center', ha='center')

    plt.title("Action Thought")
    plt.ylabel("Predicted Action")
    plt.show()

In [ ]:
plot_conf_mat(conf_mat)

# trained KNN, log_reg, GBC
# all trash, acc < 0.35